In [61]:
#Set-ExecutionPolicy -Scope Process -ExecutionPolicy Bypass
#MolGen/Scripts/activate

In [62]:
"""%pip install torch
%pip install rdkit
%pip install pandas
%pip install numpy"""

'%pip install torch\n%pip install rdkit\n%pip install pandas\n%pip install numpy'

In [5]:
import urllib.request

url = "https://raw.githubusercontent.com/aspuru-guzik-group/selfies/refs/heads/master/examples/vae_example/datasets/dataJ_250k_rndm_zinc_drugs_clean.txt"

with urllib.request.urlopen(url) as response:
    data = response.read().decode('utf-8')
    Corpus = data.strip().split('\n')

print(f"Se cargaron {len(Corpus)} moléculas.")
print("Primeras 5 moléculas:", Corpus[:5])

Se cargaron 249456 moléculas.
Primeras 5 moléculas: ['CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1', 'C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1', 'N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)cc2)cc1', 'CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c2CCCCC3)C1', 'N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#N)C12CCCCC2']


In [2]:
Dataset = ["CC(=O)O", "C1=CC=CC=C1", "C(C(=O)O)N"]
Tokens1 = []
for i in Dataset:
    Tokens1.append(list(i))
Tokens1

[['C', 'C', '(', '=', 'O', ')', 'O'],
 ['C', '1', '=', 'C', 'C', '=', 'C', 'C', '=', 'C', '1'],
 ['C', '(', 'C', '(', '=', 'O', ')', 'O', ')', 'N']]

In [3]:
Moleculas = ["CC(=O)O", "C1=CC=CC=C1", "C(C(=O)O)N"]
Tokens = []
#for i in Moleculas:
for i in Moleculas:
    for x in i:
        Tokens.append(x)
Tokens = set(Tokens)
Tokens


{'(', ')', '1', '=', 'C', 'N', 'O'}

In [4]:
#Crear vocabulario como diccionario, dandole como indice de 2 para arriba
Vocabulario = {"<pad>": 0, "<bos>": 1,} #padding e inicio de secunecia
Vocabulario["<eos>"] = len(Tokens) + 2 # Final de Secuencia. (Preguntar a Chona)
indice = 2
for i in Tokens:
    Vocabulario[i] = indice
    indice += 1
Vocabulario

{'<pad>': 0,
 '<bos>': 1,
 '<eos>': 9,
 '=': 2,
 ')': 3,
 'C': 4,
 'N': 5,
 '(': 6,
 '1': 7,
 'O': 8}

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class MolDataset(Dataset):
    def __init__(self, mols, vocab, max_len = 20): # Guarda los datos para utilizarlos en futuras funciones
        self.moleculas = mols
        self.vocabulario = vocab
        self.max_len = max_len # El len es para completar con el padding si faltan caracteres
    
    def __len__(self): # Para saber la cantidad de moleculas
        return len(self.moleculas)
    
    def __getitem__(self, nro):
        molecula = self.moleculas[nro]
        tokens_molecula = ["<bos>"] + list(molecula) + ["<eos>"] # Tokeniza a molecula seleccionada agregando la token de inicio y de final

        tokenizado_vocabulario = [self.vocabulario[t] for t in tokens_molecula] #Convertimos a los numeros del vocabulario (1.....9)

        padding = [self.vocabulario["<pad>"]] * (self.max_len - len(tokenizado_vocabulario)) # Definimos la cantidad de ceros que van a ver, la diferencia entre el maximo y las que hay.
        tokenizado_vocabulario += padding # Agregamos el padding (1.....9000)
        return torch.tensor(tokenizado_vocabulario, dtype = torch.long), molecula

In [8]:
data = MolDataset(Moleculas, Vocabulario) #Se corre la clase para mis moleculas y vocabulario

In [9]:
dataloader = DataLoader(data, batch_size = 1, shuffle = True) # Usamos dataloader para dar en tandas de a 2 (batch) y para que mezcle las moleculas tokenizadas (shuffle)

In [10]:
for batch in dataloader:
    print(batch)

[tensor([[1, 4, 7, 5, 4, 4, 5, 4, 4, 5, 4, 7, 9, 0, 0, 0, 0, 0, 0, 0]]), ('C1=CC=CC=C1',)]
[tensor([[1, 4, 4, 8, 5, 2, 3, 2, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), ('CC(=O)O',)]
[tensor([[1, 4, 8, 4, 8, 5, 2, 3, 2, 3, 6, 9, 0, 0, 0, 0, 0, 0, 0, 0]]), ('C(C(=O)O)N',)]


In [11]:
def CrearTensor(Mols, Vocab):
    data = MolDataset(Mols, Vocab) #Se corre la clase para mis moleculas y vocabulario
    dataloader = DataLoader(data, batch_size = 1, shuffle = True) # Usamos dataloader para dar en tandas de a 2 (batch) y para que mezcle las moleculas tokenizadas (shuffle)
    for batch in dataloader:
        print(batch)

In [12]:
CrearTensor(Moleculas, Vocabulario)

[tensor([[1, 4, 8, 4, 8, 5, 2, 3, 2, 3, 6, 9, 0, 0, 0, 0, 0, 0, 0, 0]]), ('C(C(=O)O)N',)]
[tensor([[1, 4, 7, 5, 4, 4, 5, 4, 4, 5, 4, 7, 9, 0, 0, 0, 0, 0, 0, 0]]), ('C1=CC=CC=C1',)]
[tensor([[1, 4, 4, 8, 5, 2, 3, 2, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), ('CC(=O)O',)]
